## Preprocessing

In [46]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense


#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [47]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"], axis = 1)

In [48]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [49]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df["APPLICATION_TYPE"].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [50]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_cutoff = 350
application_types_to_replace=app_counts[app_counts<app_cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [51]:
# Look at CLASSIFICATION value counts for binning
class_counts= application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [52]:
# You may find it helpful to look at CLASSIFICATION value counts >1
multiple_class_counts= class_counts[class_counts>1]
multiple_class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [53]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_cutoff = 1500
classifications_to_replace= class_counts[class_counts<class_cutoff].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [54]:
# Convert categorical data to numeric with `pd.get_dummies`
columns = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df = pd.get_dummies(application_df, columns=columns)

In [55]:
# Split our preprocessed data into our features and target arrays
y= application_df['IS_SUCCESSFUL']
X= application_df.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train,X_test, y_train,y_test= train_test_split(X,y)

In [56]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [57]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units= 80, activation='sigmoid', input_dim= X_train.shape[1]))

# Second hidden layer
nn.add(Dense(units= 30, activation='sigmoid'))

# Output layer
nn.add(Dense(units= 1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3520      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
# Compile the model
nn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [59]:
# Train the model
nn.fit(X_train_scaled,y_train, validation_data=(X_test_scaled, y_test), epochs= 100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.2021 - accuracy: 0.7058 - val_loss: 0.1976 - val_accuracy: 0.7217
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.1928 - accuracy: 0.7275 - val_loss: 0.1963 - val_accuracy: 0.7176
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.1912 - accuracy: 0.7279 - val_loss: 0.1938 - val_accuracy: 0.7221
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.1888 - accuracy: 0.7292 - val_loss: 0.1922 - val_accuracy: 0.7205
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.1874 - accuracy: 0.7288 - val_loss: 0.1919 - val_accuracy: 0.7241
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.1865 - accuracy: 0.7310 - val_loss: 0.1915 - val_accuracy: 0.7209
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.1859 - accuracy: 0.7297 - val_loss: 0.1904 - val_accuracy: 0.7256

In [60]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.1891 - accuracy: 0.7229 - 228ms/epoch - 850us/step
Loss: 0.18909946084022522, Accuracy: 0.7229154706001282


In [62]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")